In [1]:
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
import cartopy
import matplotlib.path as mpath
from functions import compute_ivt,to_nc
from matplotlib import rc,animation
from matplotlib.animation import FuncAnimation
from IPython import display

In [2]:
years = np.arange(2015,2100,1).astype(int)

In [3]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

In [4]:
cat = col.search(source_id=['NorESM2-LM'], experiment_id=['ssp585'], table_id=['day'], variable_id=['hus','va'], member_id=['r1i1p1f1'])

In [5]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'use_cftime':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [11]:
dataset_list = list(dset_dict.keys())
dataset_list

['ScenarioMIP.NCC.NorESM2-LM.ssp585.day.gn']

In [15]:
dset = dset_dict[dataset_list[0]]
dset

<xarray.Dataset>
Dimensions:    (member_id: 1, time: 31390, plev: 8, lat: 96, lon: 144, bnds: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(144, 2), meta=np.ndarray>
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * time       (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(15695, 2), meta=np.ndarray>
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: bnds
Data variables:
    hus        (member_id, time, plev, lat, lon) float32 dask.array<chunksize=(1, 164, 8, 96, 144), meta=np.ndarray>
    va         (member_id, time, plev, lat, lon) float32 dask.array<chunksize=(1, 143, 8, 96, 144), meta=np.ndarray>
Attributes: (12/50)
    history:                   2019-12-19T08:42:03Z ; CMOR rewrote data to be...
    mip_era:                   CMIP6
    sub_experiment_id:         none
    table_id:                  day
    physics_index:             1
    frequency:                 day
    ...                        ...
    activity_id:               ScenarioMIP
    branch_time_in_parent:     60225.0
    license:                   CMIP6 model data produced by NCC is licensed u...
    branch_time_in_child:      0.0
    parent_time_units:         days since 1850-01-01
    intake_esm_dataset_key:    ScenarioMIP.NCC.NorESM2-LM.ssp585.day.gn

In [8]:
def compute_ivt1(ivx):
    g = 9.81
    iv_ = -1/g*ivx.qv.integrate(coord='plev')
    return iv_

def floodfill(indata):
    outloc = np.copy(indata)
    for x in range(indata.shape[0]):
        # for each latitude get limit longitudes of shapes
        ind_lim = np.where(indata[x,:]==1)
        for ii in ind_lim:
            if len(ii)>1: # if there is a shape at that longitude
                jj=0
                while jj<=len(ii)-2:
                    if abs(ii[jj]-ii[jj+1])>10: # avoid filling where only 1 point in shape
                        jj=jj+1
                    else:
                        outloc[x,ii[jj]:ii[jj+1]]=1
                        jj=jj+1
    return outloc

In [17]:
# pre-computed 98th percentile IVT
q98 = xr.open_dataset('q93_2000.nc')
q98 = q98.rename({'__xarray_dataarray_variable__':'ivt'})

for year in years:
    dset = dset_dict[dataset_list[0]]
    dset = dset.sel(member_id='r1i1p1f1',time=slice(str(year)+"-01-01", str(year)+"-12-31"))
    vas = dset.va
    hus = dset.hus
    plev = dset.plev
    lat_ = hus.lat
    lon_ = hus.lon
    dset['qv'] = dset.va*dset.hus
    qv = dset.qv
    qv = xr.where(qv.plev>=25000,qv,0)
    dset['qv'] = qv
    ivt = compute_ivt1(dset)
    dset.close()
    ivt_ns = ivt.copy()
    ivt_ns = xr.where(ivt_ns.lat<0,-ivt_ns,ivt_ns,True) # minus for southern hemisphere (positive toward the pole)
    ivt_ns_pos = xr.where(ivt_ns<0,ivt_ns*0,ivt_ns,True) # negative values = not poleward
    excess = ivt_ns_pos-q98
    ivt_ns_pos.close()
    ar_points = xr.where(excess>0,1,0)
    out_ar = ar_points.copy()
    ar_points.close()
    out_ar = out_ar.drop_vars(['quantile','member_id'])
    out_loc = np.zeros((out_ar.ivt.shape[0],out_ar.ivt.shape[1],out_ar.ivt.shape[2])).astype(int)
    test_val = out_ar.ivt.values[:]
    for tt in range(len(out_ar.time)):
        df_loc = test_val[:,tt,:]
        ll = plt.contour(df_loc,levels=[0,1])
        plt.close()
        for item in ll.collections:
            for i in item.get_paths():
                v = i.vertices
                crit = abs(np.max(v[:, 1])-np.min(v[:, 1]))
                if (crit>=20): # AR has to be at least 20 deg lat 
                    xx=(v[:, 0]).astype(int)
                    yy=(v[:, 1]).astype(int)
                    for (x,y) in zip(xx,yy):
                        out_loc[y,tt,x] = 1
                        
    out_ar.ivt.values = out_loc.astype(bool)
    out_ar.to_netcdf(str(year)+'_crit_p93_ssp585.nc')
    out_ar.close()
    AR = xr.open_dataset(str(year)+'_crit_p93_ssp585.nc')
    ivt_ = AR.ivt
    ivt = xr.concat([ivt_,ivt_[:,:,:5]],dim='lon')# add extra points for AR at the edge of the world
    out_ar = ivt.copy()
    out_ar.values[:] = out_ar.values[:]*0.0

    for k in range(ivt.shape[0]):
        matrix = ivt[k,:,:]
        tst = floodfill(matrix)
        out_ar[k,:,:] = tst
    
    # add the values of the extra points to get ARs at the end of the world
    out_ar[:,:,:5] = out_ar[:,:,:5]+out_ar[:,:,144:]
    out_ar = out_ar[:,:,:144]
    out_ar = xr.where(out_ar>1,1,out_ar)
    out_ar = out_ar.astype(bool)
    lat_ = AR.lat
    lon_ = AR.lon
    out_ar.to_netcdf(str(year)+'_AR_detection_p93_ssp585.nc')

/tmp/ipykernel_6974/1899847377.py:32: UserWarning: No contour levels were found within the data range.
  ll = plt.contour(df_loc,levels=[0,1])
/tmp/ipykernel_6974/1899847377.py:32: UserWarning: No contour levels were found within the data range.
  ll = plt.contour(df_loc,levels=[0,1])
/tmp/ipykernel_6974/1899847377.py:32: UserWarning: No contour levels were found within the data range.
  ll = plt.contour(df_loc,levels=[0,1])
/tmp/ipykernel_6974/1899847377.py:32: UserWarning: No contour levels were found within the data range.
  ll = plt.contour(df_loc,levels=[0,1])
/tmp/ipykernel_6974/1899847377.py:32: UserWarning: No contour levels were found within the data range.
  ll = plt.contour(df_loc,levels=[0,1])
/tmp/ipykernel_6974/1899847377.py:32: UserWarning: No contour levels were found within the data range.
  ll = plt.contour(df_loc,levels=[0,1])
/tmp/ipykernel_6974/1899847377.py:32: UserWarning: No contour levels were found within the data range.
  ll = plt.contour(df_loc,levels=[0,1])